In [ ]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
# Schedules
[name for name in dir(keras.optimizers.schedules) if not name.startswith('_')]

['CosineDecay',
 'CosineDecayRestarts',
 'ExponentialDecay',
 'InverseTimeDecay',
 'LearningRateSchedule',
 'PiecewiseConstantDecay',
 'PolynomialDecay',
 'deserialize',
 'serialize']

In [ ]:
# 1. Polynomial Decay
lr_poly = tf.keras.optimizers.schedules. PolynomialDecay(
    initial_learning_rate=0.1, 
    decay_steps=1000, 
    end_learning_rate=0.0001, 
    power=0.5, 
    cycle=False)

In [ ]:
# 2. Exponential Decay
Ir_exp = tf.keras.optimizers.schedules. ExponentialDecay(
    initial_learning_rate=0.1, 
    decay_steps=100000,
    decay_rate=0.96)

In [ ]:
# 3. Piecewise Constant Decay
lr_ped = keras.optimizers.schedules.PiecewiseConstantDecay(
    boundaries= [100000, 110000],# s值
    values= [1.0, 0.5, 0.1])
#當訓練次數<100000, values代入 1.0
#介於100000, 110000, values代入 0.5
#>100000, values代入 0.11

In [ ]:
# Ex 練習
# Load data
from tensorflow. keras.datasets import fashion_mnist

(x_train_set, y_train_set), (x_test, y_test) = fashion_mnist.load_data()

#Split data
from sklearn.model_selection import train_test_split 

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_set, y_train_set, random_state=1)

# Preprocessing
#(45000,28,28)->(1,28,28) / keepdims=True 保持原維度，否則會變成二維(28,28)
pixel_means = x_train.mean (axis=0, keepdims=True)
pixel_stds = x_train.std(axis=0, keepdims=True)

x_train_scaled = (x_train - pixel_means) / pixel_stds
x_valid_scaled = (x_valid - pixel_means) / pixel_stds 
x_test_scaled = (x_test - pixel_means) / pixel_stds

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
tf.keras.backend.clear_session ()
np. random. seed (1)
tf.random. set_seed(1)

In [ ]:
model = keras.models.Sequential([
    keras. layers. Flatten(input_shape= [28, 28]), 
    keras. layers. Dense (300, activation='selu', kernel_initializer='lecun_normal'), 
    keras. layers. Dense (100, activation='selu', kernel_initializer='lecun_normal'), 
    keras. layers. Dense (10, activation='softmax')
])

In [ ]:
lr_exp = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1, 
    decay_steps=100000, 
    decay_rate=0.96)

model. compile(loss='sparse_categorical_crossentropy',
               optimizer=tf.keras.optimizers.SGD(learning_rate=lr_exp), 
               metrics= ['accuracy'])


In [ ]:
train = model. fit(x_train_scaled, y_train, epochs=2,
                   validation_data=(x_valid_scaled, y_valid))


Epoch 1/2
1407/1407 [==============================] - 8s 5ms/step - loss: 0.5150 - accuracy: 0.8205 - val_loss: 0.6308 - val_accuracy: 0.7771
Epoch 2/2
1407/1407 [==============================] - 6s 4ms/step - loss: 0.3824 - accuracy: 0.8611 - val_loss: 0.4549 - val_accuracy: 0.8412
